In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, f1_score, classification_report

import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [3]:
num_class =3

new_label={
    0:0,
    1:1,
    2:1,
    3:2,
    4:2
        }

path = "/kaggle/input/kneeoa/"

image_path = []
labels = []


for label in [0,2,3,4]:
    
    
    image_list = os.listdir(f"{path}train/{label}")

    for p in image_list:
        image_path+= [ f"{path}train/{label}/"+ p ]

    labels += [new_label[label]] * len(image_list)
#     labels += [label] * len(image_list)
    
train_data=pd.DataFrame({"Filepath":image_path,"Labels":labels})
train_data.shape


(4732, 2)

In [4]:
image_path = []
labels = []


for label in [0,2,3,4]:
    
    
    image_list = os.listdir(f"{path}val/{label}")

    for p in image_list:
        image_path+= [ f"{path}val/{label}/"+ p ]

    labels += [new_label[label]] * len(image_list)
    
val_data=pd.DataFrame({"Filepath":image_path,"Labels":labels})
val_data.shape


(673, 2)

In [5]:
class custom_dataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]
        image = Image.open(img_name)
        label =  self.df.iloc[idx, 1]

        image = self.transform(image)

        return image, label


In [6]:
pre_processing = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((256,256)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomResizedCrop((224,224)),
        transforms.RandomHorizontalFlip(),
#         transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'eval': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((256,256)),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [7]:
custom_train_data=custom_dataset(dataframe=train_data,transform=pre_processing['train'])
custom_val_data=custom_dataset(dataframe=val_data,transform=pre_processing['eval'])

batch_size=64
train_loader = DataLoader(custom_train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(custom_val_data, batch_size=batch_size)



In [8]:
pip install coral_pytorch

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torchvision.models as models
import torch.nn as nn
from coral_pytorch.losses import corn_loss
from coral_pytorch.dataset import corn_label_from_logits

# Using a pre-trained EfficientNetV2L model
model = models.efficientnet_v2_s(weights = 'IMAGENET1K_V1')
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_class)


# model = models.resnet34(weights='IMAGENET1K_V1')
# model.fc = nn.Linear(model.fc.in_features, num_class)

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

criterion = nn.CrossEntropyLoss()
# criterion = corn_loss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  
# scheduler = StepLR(optimizer,step_size=5)
scheduler = ReduceLROnPlateau(optimizer,mode ='min',patience =4,factor=0.1,verbose=True)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 89.4MB/s]


In [10]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = model.to(device)
print(device)

cuda


In [11]:
# def calculate_validation_loss(model, val_loader, criterion):
#     model.eval()
#     total_loss = 0.0
#     num_samples = 0

#     with torch.no_grad():
#         for inputs, targets in val_loader:
#             inputs, targets = inputs.to(device), targets.to(device)  
#             outputs = model(inputs)  
#             loss = criterion(outputs, targets)  
#             total_loss += loss.item() * inputs.size(0)
#             num_samples += inputs.size(0)

#     model.train()  
#     return total_loss / num_samples

In [12]:
from torch.cuda.amp import autocast, GradScaler
scaler=GradScaler()

In [13]:
# scaler=GradScaler()
num_epochs = 50  # the number of training epochs 
i =0
best_model_params_path = os.path.join('/kaggle/working/', 'best_model_params.pt')

torch.save(model.state_dict(), best_model_params_path)
best_acc = 0.0
for epoch in range(num_epochs):
    k =0
    print("Epoch =",i,"\n")
    model.train()  
    for inputs, labels in train_loader:
        if k%20==0:
            print(k,"/",i)
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs)
#             loss = corn_loss(outputs, labels, num_class)
            loss=criterion(outputs,labels)
#         outputs = model(inputs)
#         loss = corn_loss(outputs, labels, num_class)
        
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
#         loss.backward()
#         optimizer.step()
        
        k+=1

    i+=1
    print("=================================")
    model.eval()  
    with torch.no_grad():
        correct = 0
        total = 0
        total_loss=0
        num_samples=0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
#             loss = corn_loss(outputs, labels, num_class)  
            loss=criterion(outputs,labels)
            total_loss += loss.item() * inputs.size(0)
            num_samples += inputs.size(0)
            
            _, predicted = torch.max(outputs.data, 1)
#             predicted=corn_label_from_logits(outputs)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    validation_accuracy = correct / total

    print(f'Validation Accuracy: {validation_accuracy * 100:.2f}%')
        
    if validation_accuracy>best_acc:
        best_acc=validation_accuracy
        torch.save(model.state_dict(), best_model_params_path)

#         validation_loss = calculate_validation_loss(model, val_loader,criterion)
    avg_loss=total_loss/num_samples
    
    scheduler.step(avg_loss)
#     scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch}: Learning Rate = {current_lr}")
#         if current_lr < 5e-8:
#             break
    #     if validation_accuracy>=0.98:
    #         print("Accuracy reached")
    #         break
    print("=================================")

Epoch = 0 

0 / 0
20 / 0
40 / 0
60 / 0
Validation Accuracy: 62.26%
Epoch 0: Learning Rate = 0.0001
Epoch = 1 

0 / 1
20 / 1
40 / 1
60 / 1
Validation Accuracy: 74.15%
Epoch 1: Learning Rate = 0.0001
Epoch = 2 

0 / 2
20 / 2
40 / 2
60 / 2
Validation Accuracy: 73.70%
Epoch 2: Learning Rate = 0.0001
Epoch = 3 

0 / 3
20 / 3
40 / 3
60 / 3
Validation Accuracy: 77.86%
Epoch 3: Learning Rate = 0.0001
Epoch = 4 

0 / 4
20 / 4
40 / 4
60 / 4
Validation Accuracy: 79.79%
Epoch 4: Learning Rate = 0.0001
Epoch = 5 

0 / 5
20 / 5
40 / 5
60 / 5
Validation Accuracy: 80.98%
Epoch 5: Learning Rate = 0.0001
Epoch = 6 

0 / 6
20 / 6
40 / 6
60 / 6
Validation Accuracy: 80.39%
Epoch 6: Learning Rate = 0.0001
Epoch = 7 

0 / 7
20 / 7
40 / 7
60 / 7
Validation Accuracy: 80.24%
Epoch 7: Learning Rate = 0.0001
Epoch = 8 

0 / 8
20 / 8
40 / 8
60 / 8
Validation Accuracy: 79.35%
Epoch 8: Learning Rate = 0.0001
Epoch = 9 

0 / 9
20 / 9
40 / 9
60 / 9
Validation Accuracy: 82.02%
Epoch 9: Learning Rate = 0.0001
Epoch = 10

In [14]:
image_path = []
labels = []


for label in [0,2,3,4]:
    
    
    image_list = os.listdir(f"{path}test/{label}")

    for p in image_list:
        image_path+= [ f"{path}test/{label}/"+ p ]

    labels += [new_label[label]] * len(image_list)
    
test_data=pd.DataFrame({"Filepath":image_path,"Labels":labels})

custom_test_data=custom_dataset(dataframe=test_data,transform=pre_processing['eval'])
test_loader=DataLoader(custom_test_data,batch_size=batch_size)


In [15]:
model.load_state_dict(torch.load(best_model_params_path))
model.eval()  
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


    test_accuracy = correct / total

    print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Test Accuracy: 84.04%
